# nnUNet
Prepare input data to launch nnUNet into the CHUV cluster.

Check this repo: https://github.com/PeterMcGor/nnUNet/tree/docker_and_singularity

## Preparation

Read subjects from txt (train, validation, test)

In [ ]:
import os, sys, shutil, random, tempfile, glob, logging, math, csv

txt_dir = f'/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/'
data_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/atlas_registration/ANTs/' # train and val

# Train subjects
train_file = txt_dir + 'train_subjects.txt' # train subjects list file
train_subs = [] # empty list to read list from a file
# open file and read the content in a list
with open(train_file, 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]
        # add current item to the list
        train_subs.append(x)
# display list
print(train_subs)

# Val subjects
val_file = txt_dir + 'val_subjects.txt' # val subjects list file
val_subs = [] # empty list to read list from a file
# open file and read the content in a list
with open(val_file, 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]
        # add current item to the list
        val_subs.append(x)
# display list
print(val_subs)

# Test
file = txt_dir + 'test_subjects.txt' # test subjects list file
test_subs = [] # empty list to read list from a file
# open file and read the content in a list
with open(file, 'r') as fp:
    for line in fp:
        # remove linebreak from a current name
        # linebreak is the last character of each line
        x = line[:-1]
        # add current item to the list
        test_subs.append(x)
# display list
print(test_subs)

Copy subjects to output folder

In [ ]:
# Output paths
output_dir = f'/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet_raw_data/Task313_Eye/'
imagesTr_path = f'{output_dir}imagesTr/'
labelsTr_path = f'{output_dir}labelsTr/'
imagesTs_path = f'{output_dir}imagesTs/'

# Adding val subjects into train set for the nnUNet
train_val_subs = train_subs + val_subs

# Lengths of sets
len_test_subs = len(test_subs)
print(f'Number of test subjects: {len_test_subs}')
len_train_val_subs = len(train_val_subs)
print(f'Number of train subjects: {len_train_val_subs}')

# Toggle one subject (to test) or the entire set
TOGGLE_ENTIRE_SET = True

# Test set
for j in range(len_test_subs):
    
    # Input path
    input_t1 = data_dir + 'a123/' + test_subs[j] + '/input/' + test_subs[j] + '_T1_aff.nii.gz'
    print(f'Path T1: {input_t1}')

    # New name
    new_name_im = f'AEye_{j+1:03d}_0000.nii.gz'
    print(f'New name image T1: {new_name_im}')

    # Copy
    shutil.copy(input_t1, f'{imagesTs_path}{new_name_im}')

    if j==0 and TOGGLE_ENTIRE_SET==False:
        break

# Train set
for i in range(len_train_val_subs):
    
    # Input paths
    input_t1 = data_dir + 'a123/' + train_val_subs[i] + '/input/' + train_val_subs[i] + '_T1_aff.nii.gz'
    print(f'Path T1: {input_t1}')
    input_labels = data_dir + 'a123/' + train_val_subs[i] + '/input/' + train_val_subs[i] + '_labels_aff.nii.gz'
    print(f'Path labels: {input_labels}')

    # New name
    new_name_im = f'AEye_{i+1+len_test_subs:03d}_0000.nii.gz'
    print(f'New name image T1: {imagesTr_path}{new_name_im}')
    new_name_lab = f'AEye_{i+1+len_test_subs:03d}.nii.gz'
    print(f'New name label: {labelsTr_path}{new_name_lab}')

    # Copy
    shutil.copy(input_t1, f'{imagesTr_path}{new_name_im}')
    shutil.copy(input_labels, f'{labelsTr_path}{new_name_lab}')

    if i==0 and TOGGLE_ENTIRE_SET==False:
        break

# Dealing with files in that folder
for f in glob.glob(f'{imagesTr_path}/*0001.nii.gz'):
    os.remove(f)

Generate dataset json

In [ ]:
from nnunet.dataset_conversion.utils import generate_dataset_json

# Output paths
output_dir = f'/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet_raw_data/Task313_Eye/'
imagesTr_path = f'{output_dir}imagesTr/'
labelsTr_path = f'{output_dir}labelsTr/'
imagesTs_path = f'{output_dir}imagesTs/'

generate_dataset_json(output_file=os.path.join(output_dir, 'dataset.json'), imagesTr_dir=imagesTr_path, imagesTs_dir=imagesTs_path,
                        modalities=('T1w',), labels={0: 'background', 1: 'lens', 2: 'globe', 3: 'optic nerve',
                        4: 'intraconal fat', 5: 'extraconal fat', 6: 'lateral rectus muscle', 
                        7: 'medial rectus muscle', 8: 'inferior rectus muscle', 9: 'superior rectus muscle'},
                        dataset_name='AEyeDataset')

## Simple sample

Docker nnUnet local

In [ ]:
# Use decathlon datasets
sudo docker run --gpus device=0 --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_convert_decathlon_task -i /home/jaimebarranco/Desktop/nnUNet/nnUNet_raw_data/Task02_Heart -p NUM_PROCESSES (8 for example)

# Verify dataset integrity
sudo docker run --gpus device=0 --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_plan_and_preprocess -t 2 --verify_dataset_integrity
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_plan_and_preprocess -t 313 --verify_dataset_integrity

# To enter the Docker machine (-it for interactive and bash for the machine)
sudo docker run --gpus device=0 -it --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnUNet_resources petermcgor/nnunet:0.0.1 bash

# Training locally
sudo docker run --gpus device=0 --shm-size=10gb -v /home/jaimebarranco/Desktop/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_train 3d_fullres nnUNetTrainerV2 TaskXXX_MYTASK FOLD --npz
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_train 3d_fullres nnUNetTrainerV2 Task313_Eye 0

CHUV cluster

In [ ]:
# Login
ssh -o "ServerAliveInterval 60" ja4922@hpc1-login.chuv.ch

# Transfer files to CHUV cluster
scp -r /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnunet_slurm.sh ja4922@hpc1-login.chuv.ch:/home/ja4922/AEye/

# Transfer files from CHUV cluster
scp -r ja4922@hpc1-login.chuv.ch:/data/bach/AEye/nnUNet/nnUNet_trained_models/nnUNet/3d_fullres/Task313_Eye/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/progress.png /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/

# Training on HPC
sbatch jobfile
sbatch /home/ja4922/AEye/nnunet_slurm.sh

jobfile

In [ ]:
#!/bin/bash
#SBATCH --job-name=nn_bs2_f0
#SBATCH --chdir=/home/ja4922/AEye
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jaime.barranco-hernandez@chuv.ch
#SBATCH --ntasks=1
#SBATCH --time=1-23:59:00
#SBATCH --output=eye_3d_f0_out_%N.%j.%a.out
#SBATCH --error=eye_3d_f0_err_%N.%j.%a.err
#SBATCH --cpus-per-task=10
#SBATCH --mem=64gb
## Apparently these lines are needed for GPU execution
#SBATCH --account rad
#SBATCH --partition rad
#SBATCH --gres=gpu:1

singularity run --bind /data/bach/AEye/nnUNet:/opt/nnunet_resources --nv docker://petermcgor/nnunet:0.0.1 nnUNet_train 3d_fullres nnUNetTrainerV2 Task313_Eye 0 --npz

Running inference (on HPC): create jobfile and send it to the cluster

In [ ]:
#!/bin/bash
#SBATCH --job-name=nn_inf
#SBATCH --chdir=/home/ja4922/AEye
#SBATCH --mail-type=ALL
#SBATCH --mail-user=jaime.barranco-hernandez@chuv.ch
#SBATCH --ntasks=1
#SBATCH --time=1-23:59:00
#SBATCH --output=eye_3d_inf_out_%N.%j.%a.out
#SBATCH --error=eye_3d_inf_err_%N.%j.%a.err
#SBATCH --cpus-per-task=10
#SBATCH --mem=64gb
## Apparently these lines are needed for GPU execution
#SBATCH --account rad
#SBATCH --partition rad
#SBATCH --gres=gpu:1
singularity run --bind /data/bach/AEye/nnUNet:/opt/nnunet_resources --nv docker://petermcgor/nnunet:0.0.1 nnUNet_find_best_configuration -t 313

Changing batch_size

In [ ]:
from batchgenerators.utilities.file_and_folder_operations import *
import numpy as np
# from nnunet.paths import preprocessing_output_dir

# Paths
task_name = 'Task313_Eye'
preprocessing_output_dir = f'/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet_preprocessed/'

# if it breaks upon loading the plans file, make sure to run the Task120 dataset conversion and
# nnUNet_plan_and_preprocess first!
plans_fname = join(preprocessing_output_dir, task_name, 'nnUNetPlansv2.1_plans_3D.pkl')
plans = load_pickle(plans_fname)
plans['plans_per_stage'][0]['batch_size'] = 1
# plans['plans_per_stage'][0]['patch_size'] = np.array((512, 512))
# plans['plans_per_stage'][0]['num_pool_per_axis'] = [7, 7]

# because we changed the num_pool_per_axis, we need to change conv_kernel_sizes and pool_op_kernel_sizes as well!
# plans['plans_per_stage'][0]['pool_op_kernel_sizes'] = [[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]]
# plans['plans_per_stage'][0]['conv_kernel_sizes'] = [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]]

# for a network with num_pool_per_axis [7,7] the correct length of pool kernel sizes is 7 and the length of conv
# kernel sizes is 8! Note that you can also change these numbers if you believe it makes sense. A pool kernel size
# of 1 will result in no pooling along that axis, a kernel size of 3 will reduce the size of the feature map
# representations by factor 3 instead of 2.

# save the plans under a new plans name. Note that the new plans file must end with _plans_2D.pkl!
save_pickle(plans, join(preprocessing_output_dir, task_name, 'nnUNetPlansv2.1_bs1_plans_3D.pkl'))


Run file with changed batch size

In [ ]:
# command
nnUNet_train 3d nnUNetTrainerV2 313 0 -p nnUNetPlansv2.1_bs1
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_train 3d_fullres nnUNetTrainerV2 Task313_Eye 0 -p nnUNetPlansv2.1_bs1

Run inference locally

In [ ]:
# nnUNet_find_best_configuration will print a string to the terminal with the inference commands you need to use
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_find_best_configuration -m 3d_fullres -t 313

# determine post processing. Note that you do not have to run nnUNet_determine_postprocessing if you use nnUNet_find_best_configuration. nnUNet_find_best_configuration will do that for you.
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_determine_postprocessing  -m 3d_fullres -t 313 

# inference
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_predict -i /opt/nnunet_resources/nnUNet_raw_data/Task313_Eye/imagesTs -o /opt/nnunet_resources/nnUNet_inference/nnUNet_inference_labeled_dataset -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 -t Task313_Eye

## Non-labeled dataset INFERENCE

Copy subjects to output folder

In [ ]:
import os, sys, shutil, random, tempfile, glob, logging, math, csv
from pathlib import Path

# Input path
data_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti/'

# Output paths
output_dir = f'/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_nnunet/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

test_subjects = [sub for sub in sorted(Path(data_dir).rglob('*.nii.gz'))]
print(f'num subs: {len(test_subjects)}')

# Test set
for i in range(len(test_subjects)):
    
    # Input path
    input_t1 = test_subjects[i]
    print(f'Path T1: {input_t1}')

    # New name
    sub_name = str(test_subjects[i]).split('/')[8]
    new_name_im = f'AEye_{sub_name}_0000.nii.gz'
    print(f'New name image T1: {new_name_im}')

    # Copy
    shutil.copy(input_t1, f'{output_dir}{new_name_im}')

    # if i==0:
    #     break

# Dealing with files in that folder
# for f in glob.glob(f'{imagesTr_path}/*0001.nii.gz'):
#     os.remove(f)

run nnunet inference

In [ ]:
# inference
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_predict -i /opt/nnunet_resources/nnUNet_inference/non_labeled_dataset_nifti_nnunet -o /opt/nnunet_resources/nnUNet_inference/nnUNet_inference_non_labeled_dataset -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 -t Task313_Eye

# test (changing the postprocessing.json file to enable all the classes)
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_predict -i /opt/nnunet_resources/nnUNet_inference/test/input -o /opt/nnunet_resources/nnUNet_inference/test/output -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 -t Task313_Eye

flipped images

In [8]:
import nibabel as nib
import numpy as np

# function to switch the orientation of a .nii image from left to right to right to left
def switch_orientation(image):
    nifti_image = nib.load(image)
    image_data = nifti_image.get_fdata()
    flipped_data = np.flip(image_data, axis=1)
    flipped_image = nib.Nifti1Image(flipped_data, affine=nifti_image.affine)
    return flipped_image

input_image = "/home/jaimebarranco/Desktop/2022160100001_0000.nii.gz"
output_image = switch_orientation(input_image)
nib.save(output_image, "/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/temp_inference/input/sub-01_0000.nii.gz")

In [ ]:
sudo docker run --gpus device=0 --shm-size=10gb -v /mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet:/opt/nnunet_resources petermcgor/nnunet:0.0.1 nnUNet_predict -i /opt/nnunet_resources/nnUNet_inference/temp_inference/input -o /opt/nnunet_resources/nnUNet_inference/temp_inference/output -tr nnUNetTrainerV2 -ctr nnUNetTrainerV2CascadeFullRes -m 3d_fullres -p nnUNetPlansv2.1 -t Task313_Eye